In [0]:
# Databricks notebook source
# COMMAND ----------

# 1. Define widgets to accept parameters from the job run.
dbutils.widgets.dropdown("environment", "dev", ["dev", "test", "prod"], "Environment")
environment = dbutils.widgets.get("environment")

# 2. Read the values from the widgets.
CATALOG = f"w_{environment}"
MANAGED_UC_ROOT = f"s3a://wartsila-datalake-{environment}-managed-uc/"
BRONZE_ROOT = f"s3a://wartsila-datalake-{environment}-bronze/"
SILVER_ROOT =f"s3a://wartsila-datalake-{environment}-silver/"
GOLD_ROOT = f"s3a://wartsila-datalake-{environment}-gold/"
LANDING_ROOT = f"s3a://wartsila-datalake-{environment}-landing/fingrid/"


# 3. Build and execute the SQL commands.
print(f"Creating catalog '{CATALOG}' if it does not exist...")
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG} MANAGED LOCATION '{MANAGED_UC_ROOT}'")

print(f"Creating schemas in catalog '{CATALOG}' if they do not exist...")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.bronze MANAGED LOCATION '{BRONZE_ROOT}'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.silver MANAGED LOCATION '{SILVER_ROOT}'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.gold MANAGED LOCATION '{GOLD_ROOT}'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.landing_admin MANAGED LOCATION '{LANDING_ROOT}'")


# 4. CREATE control table creation 
SCHEMA = "landing_admin"
TABLE_NAME = f"{CATALOG}.{SCHEMA}.meta_control_table"

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    source_system STRING NOT NULL,
    source_dataset_id INT NOT NULL,
    source_dataset_name STRING NOT NULL,
    last_timestamp TIMESTAMP,
    refresh_timestamp TIMESTAMP,
    target_container STRING NOT NULL,
    target_folder STRING NOT NULL,
    page_size INT DEFAULT 5000,
    load_increment_days INT DEFAULT 1,
    environment STRING DEFAULT '{environment}',
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
    is_active BOOLEAN DEFAULT true
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')
"""


# Check if records for the current environment already exist
print(f"Checking for existing records in environment: '{environment}'...")
record_count = spark.sql(f"SELECT COUNT(*) as count FROM {TABLE_NAME} WHERE environment = '{environment}'").collect()[0]['count']


# If no records exist for the environment, insert the initial values
if record_count == 0:
    print(f"No records found for '{environment}'. Inserting initial values...")
    insert_ini_values = f"""
    INSERT INTO {TABLE_NAME} (
        source_system, source_dataset_id, source_dataset_name,
        last_timestamp, refresh_timestamp, target_container, 
        target_folder, page_size, load_increment_days
    ) 
    VALUES 
    ('Fingrid', 358, 'Electricity consumption', 
    '2025-01-01T00:00:00', '2025-05-17T11:54:00', 
    'landing', 'Electricity_consumption', 5000, 1),

    ('Fingrid', 248, 'Solar power generation', 
    '2025-01-01T00:00:00', '2025-05-17T11:50:00', 
    'landing', 'fingrid_solar_power_generation', 5000, 1),

    ('Fingrid', 245, 'Wind power generation', 
    '2025-01-01T00:00:00', '2025-05-17T11:58:00', 
    'landing', 'fingrid_wind_power_generation', 5000, 1);
    """
    spark.sql(insert_ini_values)
    print("Control table created and initial values inserted successfully!")
else:
    print(f"Records for environment '{environment}' already exist. Skipping insertion.")


print("Configuration complete.")